In [13]:
import numpy as np
import pandas as pd
import eipy.ei as e
import tensorflow as tf
import pickle as pkl
from skimage.transform import resize
from importlib import reload

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.additional_ensembles import MeanAggregation, CES
from eipy.metrics import fmax_score
from sklearn.metrics import roc_auc_score, matthews_corrcoef

In [14]:
ensemble_predictors = {
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=lambda y_test, y_pred: fmax_score(y_test, y_pred)[0]),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(),
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

In [15]:
from keras import layers, models
from keras.applications import ResNet50, MobileNet, InceptionV3, DenseNet121

res_model = ResNet50(weights='imagenet', include_top=False)
for layer in res_model.layers:
    layer.trainable = False
collecting_layer = layers.GlobalAveragePooling2D()(res_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
res_model = models.Model(inputs=res_model.input, outputs=output_layer)
res_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

base_model = MobileNet(weights='imagenet', include_top=False)
x = base_model.output
for layer in base_model.layers:
    layer.trainable = False
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(1, activation='sigmoid')(x)
net_model = models.Model(inputs=base_model.input, outputs=predictions)
net_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

pretrained_model = InceptionV3(weights='imagenet', include_top=False)
for layer in pretrained_model.layers:
    layer.trainable = False
collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
v3_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
v3_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

dense_model = DenseNet121(weights='imagenet', include_top=False)
for layer in pretrained_model.layers:
    layer.trainable = False
collecting_layer = layers.GlobalAveragePooling2D()(pretrained_model.output)
output_layer = layers.Dense(1, activation='sigmoid')(collecting_layer)
dense121_model = models.Model(inputs=pretrained_model.input, outputs=output_layer)
dense121_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

base_predictors = {"ResNet" : res_model,
                   "MobileNet" : net_model,
                   "Inception" : v3_model,
                   "DenseNet" : dense121_model
}

In [16]:
metrics = {
            'f_max': fmax_score,
            'auc': roc_auc_score,
            'mcc': matthews_corrcoef
            }

In [17]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [18]:
X_train, y_train = X_train[:10000], y_train[:10000]
X_test, y_test = X_test[10000:11000], y_test[10000:11000]

In [19]:
inanimate_labels=[0,1,8,9]
def binarize_array(arr):
    mask = np.isin(arr, inanimate_labels)

    arr = np.where(mask, 0, 1)

    return arr
y_train = binarize_array(y_train)
y_test = binarize_array(y_test)

In [23]:
new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')

In [29]:
####HI OVER HERE###
reload(e)
###HEYOOOOOO#

<module 'eipy.ei' from '/home/opc/eipy/eipy/ei.py'>

In [30]:
EI = e.EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=2,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        sampling_aggregation=None,
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name="dl",
                        model_building=False,
                        )

In [31]:
EI.fit_base(X_train_resized, y_train, modality_name="images")

Training base predictors on images...
        
... for ensemble performance analysis...
157/157 [==============================] - 27s 167ms/step
(5000,)
(5000,)
(5000,)
(5000,)
{('images', 'ResNet', 0): array([0.04770542, 0.266627  , 0.24346246, ..., 0.53117293, 0.87798446,
       0.01276997], dtype=float32), ('images', 'MobileNet', 0): array([0.0840716 , 0.979748  , 0.05079518, ..., 0.5034241 , 0.3047699 ,
       0.5041346 ], dtype=float32), ('images', 'Inception', 0): array([1.0738939e-03, 6.5129734e-07, 8.6861810e-08, ..., 3.6549143e-02,
       9.9015915e-01, 1.0482647e-08], dtype=float32), ('images', 'DenseNet', 0): array([9.9996662e-01, 2.5692868e-06, 2.5484809e-01, ..., 3.9633155e-02,
       9.9015844e-01, 5.3156279e-10], dtype=float32)}
(5000,)
(5000,)
(5000,)
(5000,)
{('images', 'ResNet', 0): array([0.0217266 , 0.95454425, 0.00480974, ..., 0.9246841 , 0.13370562,
       0.89074373], dtype=float32), ('images', 'MobileNet', 0): array([0.24291267, 0.91536653, 0.4624577 , ..., 0.9

In [32]:
EI.base_summary["metrics"]

modality          images                              
base predictor  DenseNet Inception MobileNet    ResNet
f_max           0.597290  0.579945  0.782852  0.928281
auc             0.683836  0.656760  0.897688  0.985355
mcc             0.281926  0.219818  0.608792  0.879091

In [35]:
EI.fit_ensemble(ensemble_predictors=ensemble_predictors)

Analyzing ensembles: |██████████|100%


In [37]:
EI.ensemble_summary["metrics"]

,Mean,CES,S.ADAB,S.XGB,S.DT,S.RF,S.GB,S.KNN,S.LR,S.NB,S.MLP,S.SVM
f_max,0.806885,0.851262,0.929144,0.925917,0.898435,0.927879,0.931258,0.925016,0.932652,0.932734,0.932617,0.932158
auc,0.927993,0.951935,0.985294,0.983581,0.915732,0.982982,0.986111,0.971619,0.986548,0.985631,0.986470,0.973227
mcc,0.652082,0.729492,0.881637,0.874122,0.830940,0.879304,0.880852,0.874911,0.886864,0.885432,0.886146,0.885263
